In [ ]:
import liana as li
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scanpy as sc
import os
from pycrosstalker import tools as cttl
from pycrosstalker import plots as ctpl
import mudata as mu

In [ ]:
data = mu.read_h5mu("/Tutorial_ISMBECCB2025/data_vt3/second/nichesphere/2306_HMI_allmodalities.h5mu")

In [ ]:
data = data.mod['sc']

In [ ]:
data

In [ ]:
sc.pl.embedding(data,basis='X_umap_harmony',color='cell_subtype2')

In [ ]:
for i in set(data.obs['sampleType']):
    print(i)
    lr=li.method.cellphonedb(data[data.obs['sampleType']==i],
                        groupby='cell_subtype2',
                        expr_prop=0.5,
                        verbose=True,
                        resource_name='consensus',inplace=False,use_raw=False)
    lr.to_csv(f"{i}_lr_liana_consensus.csv")

for i in os.listdir("./"):
    if i.endswith('lr_liana_consensus.csv'):
        evfull = pd.read_csv(i)
        evfull = evfull.loc[:,['ligand','receptor_complex','source','target','lr_means','cellphone_pvals']]
        evfull['type_gene_A'] = 'Ligand'
        evfull['type_gene_B'] = 'Receptor'
        evfull['gene_A'] = evfull['ligand']
        evfull['gene_B'] = evfull['receptor_complex']
        evfull['MeanLR'] = evfull['lr_means']
        k = i[0:i.find('_lr_')]
        evfull = evfull.loc[list(evfull.cellphone_pvals.to_numpy()<=0.001),:]
        evfull = evfull.loc[:, ['source', 'target', 'type_gene_A', 'type_gene_B', 'gene_A', 'gene_B', 'MeanLR']].to_csv(f'{k}_lr_ready.csv')

In [ ]:
paths = {
    'CTR': "myogenic_lr_ready.csv",
    'EXP': "ischemic_lr_ready.csv"
}
output = "output/"
if not os.path.isdir(output):
    os.mkdir(output)
data = cttl.analise_LR(paths, out_path=output, org="hsa")

In [ ]:
paths.keys()

In [ ]:
ctpl.plot_volcane(data['stats']['EXP_x_CTR'])

In [ ]:
ctpl.plot.plot_cci(graph=data["graphs"]["EXP_x_CTR"],
         colors=data["colors"],
         plt_name='Comparative (Disease vs Control)',
         coords=data["coords"],
         emax= None,
         leg= False,
         low= 0,
         high= 0,
         ignore_alpha= False,
         log= False,
         efactor= 8,
         vfactor= 12,
         pg= data["rankings"]["EXP_x_CTR"]["Pagerank"]
         )